In [8]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [10]:
url = "https://mydramalist.com/contributors"
with requests.get(url) as result:
    if result.status_code != 200:
        print("something went wrong:", result.status_code, result.reason)
    
    with open("contributors.html", "w") as writer:
        writer.write(result.text)
    
    with open("contributors.html", "r") as reader:
        html_source = reader.read()

In [12]:
print(html_source[0][:50])

<


In [14]:
page = BeautifulSoup(html_source, "html.parser")

In [16]:
page.title.text

'Top Contributors - MyDramaList'

In [18]:
top_profiles = []
top_contributors = []
for link in page.find_all("a",{"class": "text-primary"}):
    top_profiles.append(link.get("href"))
# https://www.geeksforgeeks.org/extract-all-the-urls-from-the-webpage-using-python/
for string in top_profiles:
    top_contributors.append(string[9:])
print(top_contributors[:5])

['hoshiakari', 'KanashimiAngel', 'Moey', 'xxmai', 'TheButterfly']


In [197]:
url_list = []
for username in top_contributors:
    url_list.append("https://mydramalist.com/dramalist/"+str(username)+"/completed")
print(url_list[:5])

['https://mydramalist.com/dramalist/hoshiakari/completed', 'https://mydramalist.com/dramalist/KanashimiAngel/completed', 'https://mydramalist.com/dramalist/Moey/completed', 'https://mydramalist.com/dramalist/xxmai/completed', 'https://mydramalist.com/dramalist/TheButterfly/completed']


In [22]:
url_result = []
for url in url_list:
    with requests.get(url) as resp:
        url_result.append(resp)

if url_result[0].status_code != 200:
    print("something went wrong:", url_result[0].status_code, url_result[0].reason)

for i in range(100):
    with open(str(i)+".html", "w") as writer:
        writer.write(url_result[i].text)

html_source = []
for i in range(100):
    with open(str(i)+".html", "r") as reader:
        html_source.append(reader.read())

print(html_source[0][:20])

<!DOCTYPE html>
<htm


In [24]:
pages = []
for i in range(100):
    pages.append(BeautifulSoup(html_source[i], "html.parser"))

pages[0].title.text

"hoshiakari's Drama List - MyDramaList"

In [25]:
title_elements = []
titles = []
rating_elements = []
ratings = []
link_suffixes = []
users = []
for i in range(len(pages)):
    title_elements.append(pages[i].find_all("a",{"class": "title text-primary _600"}))
    rating_elements.append(pages[i].find_all("span",{"class": "score"}))
#print(title_elements[0])
#print(len(title_elements))
for j in range(len(pages)):
    for k in range(len(title_elements[j])):
        titles.append(title_elements[j][k].text.strip())
        link_suffixes.append(title_elements[j][k].get('href'))
        users.append(top_contributors[j])
for j in range(len(pages)):
    for k in range(len(rating_elements[j])):
        ratings.append(rating_elements[j][k].text.strip())

In [34]:
print(titles[:10])
print(len(titles))
print(ratings[:10])
print(len(ratings))
print(link_suffixes[:10])
print(users[:10])
print(len(users))

['0cm4', '1 in 10,000 (Act I)', '1 Litre no Namida', '1 Litre no Namida Special', '10 Promises to My Dog', '12 Suicidal Teens', '13', '13 Assassins', '17-sai no Teikoku', '1987: When the Day Comes']
78344
['4.0', '7.0', '10', '7.0', '6.0', '6.0', '7.0', '7.0', '7.0', '9.0']
78344
['/26095-0cm4', '/30815-1-in-10-000-act-i', '/2-1-litre-no-namida', '/3114-1-litre-no-namida-special', '/509-10-promises-to-my-dog', '/31308-12-suicidal-children', '/58887-thirteen', '/2088-13-assassins-2010', '/715865-17-sai-no-teikoku', '/21985-1987']
['hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari', 'hoshiakari']
78344


In [399]:
# using link_suffixes, open each page using requests and BeautifulSoup to scrape the average rating and the first genre
    # see if it makes sense to scrape multiple genres based on the models I can use
        # linear regression?
    # use a train/test split

# use 'with' for the requests to automatically close the request when the 'with' block ends

# it may take an absurbly long time to loop through 78035 pages, so consider alternative options
    # maybe can get through it once, and put then dataframe into a csv

In [185]:
data = pd.DataFrame(list(zip(users,titles,ratings,link_suffixes)), columns=['User','Title','Rating','link_suffixes'])
data['Rating'] = pd.to_numeric(data['Rating'])
data

,User,Title,Rating,link_suffixes
0,hoshiakari,0cm4,4.0,/26095-0cm4
1,hoshiakari,"1 in 10,000 (Act I)",7.0,/30815-1-in-10-000-act-i
2,hoshiakari,1 Litre no Namida,10.0,/2-1-litre-no-namida
3,hoshiakari,1 Litre no Namida Special,7.0,/3114-1-litre-no-namida-special
4,hoshiakari,10 Promises to My Dog,6.0,/509-10-promises-to-my-dog
...,...,...,...,...
78339,Somach,You're My Pet,8.0,/1691-youre-my-pet
78340,Somach,Yuuki,8.0,/1781-yuuki
78341,Somach,Yuusha Yoshihiko to Akuryo no Kagi,8.0,/5531-yuusha-yoshihiko-to-akuryo-no-kagi
78342,Somach,Yuusha Yoshihiko to Maou no Shiro,9.0,/2449-yuusha-yoshihiko-to-maou-no-shiro


In [189]:
oamen_df = pd.DataFrame({'User': ['Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen','Oamen'],
                         'Title': ['Alchemy of Souls','Alchemy of Souls Season 2: Light and Shadow','All of Us Are Dead','Bad Genius','Business Proposal',\
                                   'Crash Landing on You','D.P.','D.P. Season 2','Descendants of the Sun','Doona!','Extraordinary Attorney Woo','Flower of Evil','Goblin','Gyeongseong Creature',\
                                   'Happiness','Hotel del Luna','Itaewon Class','Kill Bok Soon','Moon Lovers: Scarlet Heart Ryeo','Moving','My Mister','My Name','Our Beloved Summer',\
                                   'Parasite','Signal','Snowdrop','Soundtrack #1','Sweet Home','The Glory','The Pirates 2: The Last Royal Treasure','The Witch: Part 1. The Subversion',\
                                   'The World of the Married','Train to Busan','Twenty-Five Twenty-One','Twinkling Watermelon','Unlocked','Us and Them','Vincenzo','Weak Hero Class 1',\
                                   'Memories of Murder','The Glory Part 2'],
                         'Rating': [9.5,9.5,7.5,9.0,4.0,8.5,9.5,9.5,6.5,8.0,8.0,9.0,8.0,6.0,7.0,6.0,8.0,8.0,6.5,8.5,9.0,9.5,9.5,5.5,8.0,9.0,8.5,6.5,6.0,5.0,3.5,3.0,5.5,9.5,8.5,5.5,7.0,7.0,6.0,\
                                    8.5,6.0],
                         'link_suffixes': ['/52939-can-this-person-be-translated','/733261-alchemy-of-souls-part-2','/59449-our-school-now','/23604-bad-genius',\
                                           '/693591-in-house-confrontation','/35729-emergency-lands-of-love','/59001-d-p-dog-day','/713549-d-p-season-2','/10904-descendants-of-the-sun',\
                                           '/723247-lee-doo-na','/705723-strange-lawyer-woo-young-woo','/54625-flower-of-evil','/18452-goblin','/700441-gyeongseong-creature',\
                                           '/690709-happiness','/32925-hotel-del-luna','/33898-itaewon-class','/705863-kill-bok-soon','/15999-moon-lovers-scarlet-heart-ryeo',\
                                           '/25560-moving','/25172-my-ajusshi','/65391-nemesis','/694231-us-that-year','/23429-parasite','/13239-signal','/62575-snowdrop',\
                                           '/713865-why-did-you-come-to-my-house','/34064-sweet-home','/685237-untitled-kim-eun-sook-project','/32760-the-pirates-2','/29367-the-witch',\
                                           '/39975-the-married-life','/11314-train-to-busan','/695149-twenty-five-twenty-one','/739603-sparkling-watermelon','/69827-smartphone','/28319-us-and-them','/61371-vincenzo','/702267-weak-hero',\
                                           '/990-memories-of-murder','/745581-the-glory-season-2']})

data = pd.concat([data,oamen_df], ignore_index=True)
data

,User,Title,Rating,link_suffixes
0,hoshiakari,0cm4,4.0,/26095-0cm4
1,hoshiakari,"1 in 10,000 (Act I)",7.0,/30815-1-in-10-000-act-i
2,hoshiakari,1 Litre no Namida,10.0,/2-1-litre-no-namida
3,hoshiakari,1 Litre no Namida Special,7.0,/3114-1-litre-no-namida-special
4,hoshiakari,10 Promises to My Dog,6.0,/509-10-promises-to-my-dog
...,...,...,...,...
78380,Oamen,Us and Them,7.0,/28319-us-and-them
78381,Oamen,Vincenzo,7.0,/61371-vincenzo
78382,Oamen,Weak Hero Class 1,6.0,/702267-weak-hero
78383,Oamen,Memories of Murder,8.5,/990-memories-of-murder


In [191]:
data.to_csv('user-item.csv',index=False)

In [193]:
data = data.pivot_table(index='User',columns='Title',values='Rating',aggfunc='mean')
data

Title,"""Hello, Stepmother""",#Alive,#Cold Game,#Full Force Hand,#HATETAG,#Manhole,#NoFilter,#Perfake,#Remolove: Futsuu no Koi wa Jado,#TBC,...,tvN O'PENing: Babel Syndrome,tvN O'PENing: Our Beautiful Summer,tvN O'PENing: Shared Office Hookup,tvN O'PENing: XX+XY,xxxHOLiC,xxxHolic,Émotion,Τhe Straits of Love and Hate,‏2020 Idol Star Athletics Championships,≥75℃
User,,,,,,,,,,,,,,,,,,,,,
10810983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7678823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7692329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7698003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,NaN
7961539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfllower,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xAyuu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xxmai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# --------------------------------------------separate section--------------------------------------------
# --------------------------------------------separate section--------------------------------------------
# --------------------------------------------separate section--------------------------------------------
# --------------------------------------------separate section--------------------------------------------
# --------------------------------------------separate section--------------------------------------------

In [199]:
df = pd.DataFrame(list(zip(titles, ratings,link_suffixes)), columns=['Title', 'Rating','link_suffixes'])
print(df[:15])

                                        Title Rating  \
0                                        0cm4    4.0   
1                         1 in 10,000 (Act I)    7.0   
2                           1 Litre no Namida     10   
3                   1 Litre no Namida Special    7.0   
4                       10 Promises to My Dog    6.0   
5                           12 Suicidal Teens    6.0   
6                                          13    7.0   
7                                13 Assassins    7.0   
8                           17-sai no Teikoku    7.0   
9                    1987: When the Day Comes    9.0   
10          2018 MBC Music Festival: The Live    6.0   
11                2022 KBS Song Festival: Y2K    7.0   
12  20th Century Boys 1: Beginning of the End    7.0   
13                                     3 Days    8.0   
14                                     3-Iron     10   

                                    link_suffixes  
0                                     /26095-0cm4  

In [201]:
df['Rating'] = pd.to_numeric(df['Rating'])
print(df.dtypes)

Title             object
Rating           float64
link_suffixes     object
dtype: object


In [550]:
# df.groupby('Title').median().reset_index()
# print(df)

In [552]:
# # Assuming 'df' is your DataFrame and 'Title' is a column you want to group by
# # We will select only numeric columns for the median calculation

# # Group by 'Title' and calculate the median for numeric columns only
# numeric_df = df.select_dtypes(include='number')  # Select only numeric columns
# result = df.groupby('Title')[numeric_df.columns].median().reset_index()  # Group and calculate median

# print(result)  # Print the result instead of the original DataFrame

In [203]:
df.loc[:, 'title_folded'] = df['Title'].astype(str).str.lower()

df

,Title,Rating,link_suffixes,title_folded
0,0cm4,4.0,/26095-0cm4,0cm4
1,"1 in 10,000 (Act I)",7.0,/30815-1-in-10-000-act-i,"1 in 10,000 (act i)"
2,1 Litre no Namida,10.0,/2-1-litre-no-namida,1 litre no namida
3,1 Litre no Namida Special,7.0,/3114-1-litre-no-namida-special,1 litre no namida special
4,10 Promises to My Dog,6.0,/509-10-promises-to-my-dog,10 promises to my dog
...,...,...,...,...
78339,You're My Pet,8.0,/1691-youre-my-pet,you're my pet
78340,Yuuki,8.0,/1781-yuuki,yuuki
78341,Yuusha Yoshihiko to Akuryo no Kagi,8.0,/5531-yuusha-yoshihiko-to-akuryo-no-kagi,yuusha yoshihiko to akuryo no kagi
78342,Yuusha Yoshihiko to Maou no Shiro,9.0,/2449-yuusha-yoshihiko-to-maou-no-shiro,yuusha yoshihiko to maou no shiro


In [205]:
sorted_df = df.sort_values(by=['title_folded'])
sorted_df['frequency'] = sorted_df.groupby('title_folded')['title_folded'].transform('count')
sorted_df

,Title,Rating,link_suffixes,title_folded,frequency
27208,"""Hello, Stepmother""",6.5,/728343-hello-stepmother,"""hello, stepmother""",1
28092,#Alive,7.0,/38787-alone,#alive,30
76085,#Alive,8.0,/38787-alone,#alive,30
53970,#Alive,0.0,/38787-alone,#alive,30
18303,#Alive,5.5,/38787-alone,#alive,30
...,...,...,...,...,...
595,‏2020 Idol Star Athletics Championships,8.5,/53045-2020-idol-star-athletics-championships,‏2020 idol star athletics championships,3
66716,‏2020 Idol Star Athletics Championships,7.5,/53045-2020-idol-star-athletics-championships,‏2020 idol star athletics championships,3
21678,≥75℃,8.0,/53419-75,≥75℃,3
76086,≥75℃,7.0,/53419-75,≥75℃,3


In [207]:
trimmed_df = sorted_df[sorted_df['frequency'] >= 15]
trimmed_df

,Title,Rating,link_suffixes,title_folded,frequency
28092,#Alive,7.0,/38787-alone,#alive,30
76085,#Alive,8.0,/38787-alone,#alive,30
53970,#Alive,0.0,/38787-alone,#alive,30
18303,#Alive,5.5,/38787-alone,#alive,30
48337,#Alive,7.0,/38787-alone,#alive,30
...,...,...,...,...,...
40172,Zombie Detective,8.0,/61063-zombie-detective,zombie detective,16
16689,Zombie Detective,8.0,/61063-zombie-detective,zombie detective,16
27206,Zombie Detective,7.5,/61063-zombie-detective,zombie detective,16
48610,Zombie Detective,8.0,/61063-zombie-detective,zombie detective,16


In [ ]:
# numeric_df = trimmed_df.select_dtypes(include='number')  # Select only numeric columns
# median_df = trimmed_df.groupby(['title_folded','link_suffixes'])[numeric_df.columns].median().reset_index()  # Group and calculate median

In [303]:
trimmed_df.loc[:,'median_rating'] = trimmed_df.groupby('title_folded')['Rating'].transform('median')

print(trimmed_df)

                  Title  Rating            link_suffixes      title_folded  \
28092            #Alive     7.0             /38787-alone            #alive   
76085            #Alive     8.0             /38787-alone            #alive   
53970            #Alive     0.0             /38787-alone            #alive   
18303            #Alive     5.5             /38787-alone            #alive   
48337            #Alive     7.0             /38787-alone            #alive   
...                 ...     ...                      ...               ...   
40172  Zombie Detective     8.0  /61063-zombie-detective  zombie detective   
16689  Zombie Detective     8.0  /61063-zombie-detective  zombie detective   
27206  Zombie Detective     7.5  /61063-zombie-detective  zombie detective   
48610  Zombie Detective     8.0  /61063-zombie-detective  zombie detective   
59586  Zombie Detective    10.0  /61063-zombie-detective  zombie detective   

       frequency  median_rating  
28092         30            7

In [382]:
median_df = trimmed_df.drop_duplicates(subset=['title_folded'])
median_df = median_df[median_df.median_rating >= 7.0]
median_df = median_df[median_df.Title != 'HIStory: Stay Away from Me']
median_df = median_df[median_df.Title != 'Present Perfect']
median_df = median_df[median_df.Title != 'Step for You']
median_df = median_df[median_df.Title != 'Tsuge & Minato']
median_df = median_df.reset_index(drop=True)
median_df = median_df.drop(columns=['Rating'])
median_df.loc[158,'link_suffixes'] = '/515-departures'
median_df

,Title,link_suffixes,title_folded,frequency,median_rating
0,#Alive,/38787-alone,#alive,30,7.0
1,1 Litre no Namida,/2-1-litre-no-namida,1 litre no namida,23,9.0
2,100 Days My Prince,/27392-hundred-days-husband,100 days my prince,25,7.5
3,100-manen no Onna-tachi,/22885-100-manen-no-onna-tachi,100-manen no onna-tachi,20,8.0
4,18 Again,/52941-eighteen-again,18 again,31,8.5
...,...,...,...,...,...
837,Your Name Engraved Herein,/36661-your-name-engraved-herein,your name engraved herein,19,8.5
838,Yu Yu Hakusho,/683555-yu-yu-hakusho,yu yu hakusho,16,8.0
839,YYY,/58613-yyy,yyy,16,7.5
840,Zettai BL ni Naru Sekai VS Zettai BL ni Narita...,/689215-absolute-bl,zettai bl ni naru sekai vs zettai bl ni narita...,24,8.0


In [383]:
median_df.to_csv('medians.csv')

In [384]:
median_df = pd.read_csv('medians.csv')
median_df = median_df.drop(median_df.columns[0],axis=1)
median_df

,Title,link_suffixes,title_folded,frequency,median_rating
0,#Alive,/38787-alone,#alive,30,7.0
1,1 Litre no Namida,/2-1-litre-no-namida,1 litre no namida,23,9.0
2,100 Days My Prince,/27392-hundred-days-husband,100 days my prince,25,7.5
3,100-manen no Onna-tachi,/22885-100-manen-no-onna-tachi,100-manen no onna-tachi,20,8.0
4,18 Again,/52941-eighteen-again,18 again,31,8.5
...,...,...,...,...,...
837,Your Name Engraved Herein,/36661-your-name-engraved-herein,your name engraved herein,19,8.5
838,Yu Yu Hakusho,/683555-yu-yu-hakusho,yu yu hakusho,16,8.0
839,YYY,/58613-yyy,yyy,16,7.5
840,Zettai BL ni Naru Sekai VS Zettai BL ni Narita...,/689215-absolute-bl,zettai bl ni naru sekai vs zettai bl ni narita...,24,8.0


In [385]:
links = []
for link in median_df['link_suffixes']:
    with requests.get('https://mydramalist.com'+str(link)) as resp:
        links.append(resp)

In [386]:
if links[0].status_code != 200:
    print("something went wrong:", links[0].status_code, links[0].reason)

for i in range(median_df.shape[0]):
    with open("_"+str(i)+".html", "w") as writer:
        writer.write(links[i].text)

html_source = []
for i in range(median_df.shape[0]):
    with open("_"+str(i)+".html", "r") as reader:
        html_source.append(reader.read())

print(html_source[0][:20])

<!DOCTYPE html>
<htm


In [387]:
pages = []
for i in range(median_df.shape[0]):
    pages.append(BeautifulSoup(html_source[i], "html.parser"))

In [388]:
pages[283].title.text

'Hit the Top (2017) - MyDramaList'

In [389]:
list_items = []
link_items = []
actor_items = []

for i in range(len(pages)):
    list_items.append(pages[i].find_all("li",{"class": "list-item p-a-0"}))
    genre_items = pages[i].find_all("li", {"class": "list-item p-a-0 show-genres"})
    for genre_item in genre_items:
        link_items.append(genre_item.find_all("a", {"class": "text-primary"}))
    actor_items.append(pages[i].find_all("b",{"itempropx": "name"}))

In [390]:
print(actor_items[158][0].text.strip())
print(actor_items[158][1].text.strip())

Motoki Masahiro
Hirosue Ryoko


In [391]:
print(link_items[0][0].text.strip())
print(link_items[0][1].text.strip())

Thriller
Mystery


In [392]:
print(list_items[0][3].text.strip()[:8])

Country:


In [393]:
print(list_items[0][3].text.strip())

Country: South Korea


In [394]:
print(list_items[0][10].text.strip()[:6])

Conten


In [418]:
actor1 = []
actor2 = []
genre1 = []
genre2 = []
country = []
score = []

for i in range(len(actor_items)):        
    actor1.append(actor_items[i][0].text.strip())
    actor2.append(actor_items[i][1].text.strip())
    genre1.append(link_items[i][0].text.strip())
    genre2.append(link_items[i][1].text.strip())
    for j in range(7):
        if (list_items[i][j].text.strip()[:8] == "Country:"):
            country.append(list_items[i][j].text.strip()[9:])
    for h in range(15):
        if (list_items[i][h].text.strip()[:6] == "Score:"):
            score.append(list_items[i][h].text.strip()[7:11])
    
    # country.append(list_items[i][4].text.strip()[9:])
    # score.append(list_items[i][11].text.strip()[7:11])

In [419]:
print(len(country))
print(len(score))

842
842


In [420]:
full_df = median_df
full_df['actor1'] = actor1
full_df['actor2'] = actor2
full_df['genre1'] = genre1
full_df['genre2'] = genre2
full_df['country'] = country
full_df['score'] = score

In [421]:
full_df

,Title,link_suffixes,title_folded,frequency,median_rating,actor1,actor2,genre1,genre2,country,score
0,#Alive,/38787-alone,#alive,30,7.0,Yoo Ah In,Park Shin Hye,Thriller,Mystery,South Korea,7.9
1,1 Litre no Namida,/2-1-litre-no-namida,1 litre no namida,23,9.0,Sawajiri Erika,Nishikido Ryo,Romance,Drama,Japan,9.0
2,100 Days My Prince,/27392-hundred-days-husband,100 days my prince,25,7.5,Doh Kyung Soo,Nam Ji Hyun,Historical,Comedy,South Korea,8.3
3,100-manen no Onna-tachi,/22885-100-manen-no-onna-tachi,100-manen no onna-tachi,20,8.0,Noda Yojiro,Matsui Rena,Thriller,Mystery,Japan,8.0
4,18 Again,/52941-eighteen-again,18 again,31,8.5,Kim Ha Neul,Yoon Sang Hyun,Romance,Life,South Korea,8.6
...,...,...,...,...,...,...,...,...,...,...,...
837,Your Name Engraved Herein,/36661-your-name-engraved-herein,your name engraved herein,19,8.5,Edward Chen,Tseng Ching Hua,Romance,Youth,Taiwan,8.2
838,Yu Yu Hakusho,/683555-yu-yu-hakusho,yu yu hakusho,16,8.0,Kitamura Takumi,Uesugi Shuhei,Action,Adventure,Japan,8.1
839,YYY,/58613-yyy,yyy,16,7.5,Yoon Phusanu Wongsavanischakorn,Lay Talay Sanguandikul,Comedy,Romance,Thailand,7.0
840,Zettai BL ni Naru Sekai VS Zettai BL ni Narita...,/689215-absolute-bl,zettai bl ni naru sekai vs zettai bl ni narita...,24,8.0,Inukai Atsuhiro,Yutaro,Comedy,Romance,Japan,7.6


In [422]:
full_df.to_csv('full.csv')

In [436]:
item_features = full_df[['actor1','actor2','genre1','genre2','country']]
item_features.index = full_df['title_folded']
item_features

,actor1,actor2,genre1,genre2,country
title_folded,,,,,
#alive,Yoo Ah In,Park Shin Hye,Thriller,Mystery,South Korea
1 litre no namida,Sawajiri Erika,Nishikido Ryo,Romance,Drama,Japan
100 days my prince,Doh Kyung Soo,Nam Ji Hyun,Historical,Comedy,South Korea
100-manen no onna-tachi,Noda Yojiro,Matsui Rena,Thriller,Mystery,Japan
18 again,Kim Ha Neul,Yoon Sang Hyun,Romance,Life,South Korea
...,...,...,...,...,...
your name engraved herein,Edward Chen,Tseng Ching Hua,Romance,Youth,Taiwan
yu yu hakusho,Kitamura Takumi,Uesugi Shuhei,Action,Adventure,Japan
yyy,Yoon Phusanu Wongsavanischakorn,Lay Talay Sanguandikul,Comedy,Romance,Thailand


In [438]:
item_features.to_csv('item_features.csv')